# コードの概要

このコードは、**「深層学習2 ― 実践的なニューラルネットワークの仕組みと実装」**という本のリポジトリ内で実装されている、**Attention機構**（注意機構）に関するPythonコードです。  
主に、ニューラルネットワーク（とくに自然言語処理や機械翻訳など）の文脈で使われる「Attention」の計算を行うクラス群です。

### 主なクラスと役割

#### 1. `WeightSum`
- **役割**: Attentionで計算した重み付き和を求めるレイヤー。
- **forward**: 入力ベクトル群 `hs` と attention重み `a` を使って、重み付き和を計算。
- **backward**: 勾配計算（逆伝播）を行う。

#### 2. `AttentionWeight`
- **役割**: Attentionの重み（重要度）を計算するレイヤー。
- **forward**: 入力ベクトル群 `hs` とクエリベクトル `h` の類似度（内積）を求め、softmaxで正規化して重み `a` を計算。
- **backward**: 勾配計算（逆伝播）。

#### 3. `Attention`
- **役割**: `AttentionWeight`と`WeightSum`を組み合わせて、Attention機構全体を実装。
- **forward**: 重み計算→重み付き和の計算という流れ。
- **backward**: 逆伝播の勾配計算。

#### 4. `TimeAttention`
- **役割**: 時系列データ（RNNなどで使う）に対応したAttentionレイヤー。
- **forward**: デコーダ側の各時刻ごとにAttentionを計算し、出力に格納。
- **backward**: 各時刻ごとに逆伝播を計算。

---

### まとめ

- **Attentionの計算手順**  
  1. 入力（エンコーダの隠れ状態など）とクエリ（デコーダの隠れ状態など）から重みを計算（`AttentionWeight`）。
  2. 重みを使って入力の重み付き和を計算（`WeightSum`）。
  3. これらを統合して、Attention層（`Attention`）として利用。
  4. 時系列データに対応するため、各時刻ごとにAttention計算（`TimeAttention`）。

- **用途**  
  主にSeq2Seqモデルなどで、入力系列のどの部分に着目すべきかを学習するために利用されます。

---

もし、「どの部分が知りたい」「具体的な処理内容」「使い方」など質問があれば、さらに詳しく解説できます。

In [ ]:
import sys
sys.path.append("..")
from common.np import *
from common.layers import Softmax

In [ ]:
class WeightSum:
    def __init__(self) -> None:
        self.params, self.grads = [], []
        self.cache = None
    
    def forward(self, hs, a):
        N, T, H = hs.shape

        ar = a.reshape(N, T, 1)
        t = hs * ar
        c = np.sum(t, axis=1)

        self.cache = (hs, ar)

        return c
    
    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        dt = dc.reshape(N, 1, H).repeat(T, axis=1)
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2)

        return dhs, da

In [ ]:
class AttentionWeight:
    def __init__(self) -> None:
        self.params, self.grads = [], []
        self.softmax = Softmax()
        self.cache = None

    def forward(self, hs, h):
        N, T, H = hs.shape

        hr = h.reshape(N, 1, H)
        t = hs * hr
        s = np.sum(t, axis=2)
        a = self.softmax.forward(s)

        self.cache = (hs, hr)
        return a
    
    def backward(self, da):
        hs, hr = self.cache
        N, T, H = hs.shape

        ds = self.softmax.backward(da)
        dt = ds.reshape(N, T, 1).repeat(H, axis=2)
        dhs = dt * hr
        dhr = dt * hs
        dh = np.sum(dhr, axis=1)

        return dhs, dh

In [ ]:
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        self.attention_weight_layer = AttentionWeight()
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh

In [ ]:

class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        self.attention_weights = []

        for t in range(T):
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec